In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import sweetviz as sv

In [2]:
PROJECT_PATH = "D:/ProjectInterPromo/"

In [3]:
def bool_other_col(data: pd.DataFrame):
    bool_col = []
    for i in data.columns:
        unique_value = np.unique(data[i])

        if len(unique_value) == 2:
            if 1 in unique_value and 0 in unique_value:
                bool_col.append(i)

    autre_col = list(set(data.columns) - set(bool_col))
    return autre_col, bool_col

In [4]:
# Transformation function

def puissance_10(X: pd.Series):
    return np.unique(np.floor(np.log10(X + 1)), return_counts=True)


def var_neg(colonne):
    for elem in colonne:
        if elem < 0:
            return True
    return False


def var_neg_affichage(colonne):
    return np.maximum(colonne, 0), np.minimum(colonne, 0)


def zeros(col, seuil):
    return (col == 0).sum() / len(col) >= seuil


def to_bool(col):
    return (col != 0).astype(int)


# df[col] = df[col].apply(replaceNonZero)

In [5]:
def transformation_pos(col):
    # Si taux 0 >> 0.9
    if zeros(col, 0.9):
        # Return to_bool
        # return np.array([i if i == 0 else 1 for i in col])
        return to_bool(col)

    # Sinon
    else:
        power = puissance_10(col)[0]

        # Application log si valeur grande, ou plus de 3 ordre différent
        if max(power) > 6 or len(power) > 2:
            return np.log(col + 1)

        # Peu d'ordre de grandeur application d'une sqrt
        elif max(power) > 0:
            return np.sqrt(col)
        
        return col


def transformation(df: pd.DataFrame):
    df2 = df.copy()
    autre_col, bool_col = bool_other_col(df)

    for col in tqdm(autre_col, total=len(autre_col)):
        if col in ["ID", "TOP_FRAUDE", "TOP_FRAUDE_VIREMENT", "TOP_FRAUDE_CARTE"]:
            pass
        else:
            if var_neg(df[col]):
                pos, neg = var_neg_affichage(df[col])
                
                # Full neg or 0
                if len(np.unique(pos)) == 1:
                    df2[col] = transformation_pos(- np.array(neg)).copy()

                else:
                    # df2[col + "_neg"] = [i if i == 0 else 1 for i in -np.array(neg)]
                    df2[col + "_pos"] = transformation_pos(np.array(pos))
                    df2[col + "_neg"] = to_bool(- np.array(neg))
                    df2 = df2.drop(columns=[col])

            else:
                df2[col] = transformation_pos(df[col])
    return df2

In [6]:
ech_train = pd.read_csv(PROJECT_PATH + "data/donnee_v2_ech_train.csv")
ech_test = pd.read_csv(PROJECT_PATH + "data/donnee_v2_ech_test.csv")

In [7]:
transfo = transformation(ech_train)

  0%|          | 0/202 [00:00<?, ?it/s]

In [8]:
transfo.to_csv(PROJECT_PATH + "data/donnee_v2_ech_train_transfo.csv")

In [9]:
autre, bool_col = bool_other_col(ech_train)
new_col = list((set(transfo.columns) - set(ech_train.columns)).union(set(transfo.columns).intersection(set(autre))))

In [10]:
len(new_col)

219

In [11]:
my_report = sv.analyze(transfo[new_col], pairwise_analysis="off")
my_report.show_html(filepath="report_transfo.html") # Default arguments will generate to "SWEETVIZ_REPORT.html"

                                             |          | [  0%]   00:00 -> (? left)

Report report_transfo.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [12]:
my_report = sv.analyze(ech_train[autre], pairwise_analysis="off")
my_report.show_html(filepath="report_raw.html") # Default arguments will generate to "SWEETVIZ_REPORT.html"

                                             |          | [  0%]   00:00 -> (? left)

Report report_raw.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
